### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

## 1. Let's look at the data we have

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
data = pd.io.stata.read_stata('eda-projects/us_job_market_discrimination.dta')
data = data[['race', 'call']]
data.shape

(4870, 2)

In [2]:
blacks = data[data.race == 'b']
blacks.call.describe()

count    2435.000000
mean        0.064476
std         0.245650
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [3]:
whites = data[data.race == 'w']
whites.call.describe()

count    2435.000000
mean        0.096509
std         0.295349
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

*In our sample*, the proportion of call-backs ("calls") for white-sounding-names ("whites") is 9.65%, while for "blacks" it's 6.44%.

## 2. Chi-squared

Is that statistically significant? Let's do a X².

In [4]:
blacks_called = len(blacks[blacks['call'] == True])
blacks_not_called = len(blacks[blacks['call'] == False])
whites_called = len(whites[whites['call'] == True])
whites_not_called = len(whites[whites['call'] == False])
observed = pd.DataFrame({'blacks': {'called': blacks_called, 'not_called': blacks_not_called},
                         'whites': {'called' : whites_called, 'not_called' : whites_not_called}})
observed

,blacks,whites
called,157,235
not_called,2278,2200


In [5]:
#polled proportion
num_called_back = blacks_called + whites_called
num_not_called = blacks_not_called + whites_not_called
rate_of_callbacks = num_called_back * 1.0 / num_not_called
rate_of_callbacks

0.08753907994640464

In [6]:
expected_called = len(data) / 2 * rate_of_callbacks
expected_not_called = len(data) / 2 * (1 - rate_of_callbacks)
print(expected_called)
print(expected_not_called)

213.157659669
2221.84234033


In [7]:
import scipy.stats as stats
observed_frequencies = [blacks_not_called, whites_not_called, whites_called, blacks_called]
expected_frequencies = [expected_not_called, expected_not_called, expected_called, expected_called]
print(observed_frequencies)
print(expected_frequencies)

[2278, 2200, 235, 157]
[2221.8423403305046, 2221.8423403305046, 213.1576596694953, 213.1576596694953]


In [8]:
# We use degrees of freedom 1, which we calculate as 
# (num columns - 1) * (num_rows - 1) = 1 
# The degrees of freedom is k - 1 - ddof (the adjustment to the degrees of freedom)
stats.chisquare(f_obs = observed_frequencies,
                f_exp = expected_frequencies, ddof=2)

Power_divergenceResult(statistic=18.667389542060633, pvalue=1.5562141908802774e-05)

Highly significant.

## 3. Margin of error, confidence interval and p-value

Compare rates with two-tailed test of proportions.

### 3.a. Compare proportions using p-value

Pooled proportions and z-statistic. Then, if p-value will be less than 0.05, proportions will be considered significantly different.

In [9]:
n1 = len(whites)
n2 = len(blacks)
p1 = whites_called * 1.0 / n1
p2 = blacks_called * 1.0 / n2
print(p1)
print(p2)
p = (p1 * n1 + p2 * n2) / (n1 + n2)
p

0.0965092402464
0.064476386037


0.08049281314168377

In [10]:
stderr = np.sqrt(p * (1 - p) * ((1.0 / n1) + (1.0 / n2)))
stderr

0.0077968940361704568

In [11]:
z = (p1 - p2) / stderr
z

4.1084121524343464

In [12]:
p_values = stats.norm.sf(abs(z))*2 #twosided
p_values

3.9838868375850767e-05

p-value is less than alpha (0.05) so the null hypothesis is rejected.

## 3.b. Confidence interval

In [13]:
p = p1 - p2
stderr = np.sqrt(p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2)
z = 1.96
confidence_interval = [p - z * stderr, p + z * stderr]
confidence_interval

[0.020255520134115839, 0.043810188284775328]

95% confidence that the true difference of proportions is within 0.02 and 0.04, i.e. different from zero.

## 4. Significance

Above, we've used 95% confidence. This means we're quite sure there is a difference, but we could be wrong 5% of the times.